In [39]:
import arcpy
import requests
import os
from arcpy.sa import *

In [8]:
#make call
url = "https://ndawn.ndsu.nodak.edu/table.csv?station=78&station=111&station=98&station=162&station=174&station=142&station=164&station=138&station=161&station=9&station=160&station=159&station=10&station=118&station=56&station=165&station=11&station=12&station=58&station=13&station=84&station=55&station=179&station=7&station=186&station=87&station=14&station=15&station=96&station=191&station=16&station=201&station=137&station=124&station=143&station=17&station=85&station=140&station=134&station=18&station=136&station=65&station=104&station=99&station=192&station=19&station=129&station=20&station=101&station=166&station=178&station=81&station=21&station=97&station=22&station=75&station=184&station=2&station=172&station=139&station=158&station=23&station=157&station=62&station=86&station=24&station=89&station=126&station=167&station=93&station=183&station=90&station=25&station=83&station=107&station=156&station=77&station=26&station=155&station=70&station=127&station=144&station=27&station=173&station=132&station=28&station=195&station=185&station=29&station=30&station=154&station=31&station=187&station=102&station=32&station=119&station=4&station=80&station=33&station=59&station=153&station=105&station=82&station=34&station=198&station=72&station=135&station=35&station=76&station=120&station=141&station=109&station=36&station=79&station=193&station=71&station=37&station=38&station=189&station=39&station=130&station=73&station=188&station=40&station=41&station=54&station=69&station=194&station=145&station=113&station=128&station=42&station=43&station=103&station=171&station=116&station=196&station=88&station=114&station=3&station=163&station=200&station=64&station=115&station=168&station=67&station=175&station=146&station=170&station=197&station=44&station=133&station=106&station=100&station=121&station=45&station=46&station=61&station=66&station=181&station=74&station=60&station=199&station=125&station=176&station=177&station=8&station=180&station=204&station=47&station=122&station=108&station=5&station=152&station=48&station=151&station=147&station=68&station=169&station=49&station=50&station=91&station=182&station=117&station=63&station=150&station=51&station=6&station=52&station=92&station=112&station=131&station=123&station=95&station=53&station=203&station=190&station=57&station=149&station=148&station=202&station=110&variable=ddavt&year=2023&ttype=daily&quick_pick=&begin_date=2024-10-25&end_date=2024-11-25"
response = requests.get(url)

In [9]:
#set folder 
data_folder = r"C:\Users\schil726\Downloads\GIS5571_Lab3\data"

In [12]:
#set path for csv and write url response to file
csv_path = r"C:\Users\schil726\Downloads\GIS5571_Lab3\data\NDAWN_tempature.csv"

with open(csv_path, "wb") as file:
    file.write(response.content)

In [15]:
NDAWN_temperature = pd.read_csv(csv_path, header = 4)
NDAWN_temperature

,Unnamed: 0,deg,deg.1,ft,Unnamed: 4,Unnamed: 5,Unnamed: 6,Degrees F,Unnamed: 8
0,Ada,47.32119,-96.51406,910,2024,10,25,42.620,NaN
1,Ada,47.32119,-96.51406,910,2024,10,26,42.809,NaN
2,Ada,47.32119,-96.51406,910,2024,10,27,46.162,NaN
3,Ada,47.32119,-96.51406,910,2024,10,28,54.805,NaN
4,Ada,47.32119,-96.51406,910,2024,10,29,51.004,NaN
...,...,...,...,...,...,...,...,...,...
6171,Zeeland,46.01351,-99.68768,2070,2024,11,21,15.338,NaN
6172,Zeeland,46.01351,-99.68768,2070,2024,11,22,22.719,NaN
6173,Zeeland,46.01351,-99.68768,2070,2024,11,23,24.432,NaN
6174,Zeeland,46.01351,-99.68768,2070,2024,11,24,19.212,NaN


In [25]:
#drop columns, clean up column names, and add temperature min and max columns
NDAWN_temperature.rename(columns={'Unnamed: 0':'Station', 'deg':'Lat', 'deg.1':'Long', 'Degrees F':'Avg Temp'}, inplace=True)
NDAWN_temperature['MAX'] = NDAWN_temperature['Avg Temp']
NDAWN_temperature['MIN'] = NDAWN_temperature['Avg Temp']
drop_columns = [3, 4, 5, 6, 8]
NDAWN_temperature = NDAWN_temperature.drop(columns=NDAWN_temperature.columns[drop_columns])
NDAWN_temperature

,Station,Latitude,Longitude,Avg Temp,MAX,MIN
0,Ada,47.32119,-96.51406,42.620,42.620,42.620
1,Ada,47.32119,-96.51406,42.809,42.809,42.809
2,Ada,47.32119,-96.51406,46.162,46.162,46.162
3,Ada,47.32119,-96.51406,54.805,54.805,54.805
4,Ada,47.32119,-96.51406,51.004,51.004,51.004
...,...,...,...,...,...,...
6171,Zeeland,46.01351,-99.68768,15.338,15.338,15.338
6172,Zeeland,46.01351,-99.68768,22.719,22.719,22.719
6173,Zeeland,46.01351,-99.68768,24.432,24.432,24.432
6174,Zeeland,46.01351,-99.68768,19.212,19.212,19.212


In [26]:
NDAWN_temperature_cleaned = NDAWN_temperature.to_csv(r"C:\Users\schil726\Downloads\GIS5571_Lab3\data\NDAWN_temperature_cleaned.csv")

In [27]:
#feature class via XY table to point
arcpy.management.XYTableToPoint(
    in_table = r"C:\Users\schil726\Downloads\GIS5571_Lab3\data\NDAWN_temperature_cleaned.csv",
    out_feature_class = r"C:\Users\schil726\Downloads\GIS5571_Lab3\data\NDAWN_stations",
    x_field = "Longitude",
    y_field = "Latitude",
)

<Result 'C:\\Users\\schil726\\Downloads\\GIS5571_Lab3\\data\\NDAWN_stations.shp'>

In [29]:
#interpolation method 1: inverse distance weighting (IDW)
arcpy.ddd.Idw(
    in_point_features = "NDAWN_stations",
    z_field = "Avg_Temp",
    out_raster = r"C:\Users\schil726\Downloads\GIS5571_Lab3\data\IDW"
)

<Result 'C:\\Users\\schil726\\Downloads\\GIS5571_Lab3\\data\\IDW'>

In [38]:
#interpolation method 2: kriging
#set variables 
lagSize = 2000
majorRange = 2.6
partialSill = 542
nugget = 0
k_model = kModelOrdinary = KrigingModelOrdinary("SPHERICAL", lagSize,
                                majorRange, partialSill, nugget)

kriging = arcpy.sa.Kriging(
    in_point_features = "NDAWN_stations",
    z_field = "Avg_Temp",
    kriging_model = k_model,
)

kriging.save = (r"C:\Users\schil726\Downloads\GIS5571_Lab3\data\kriging")

In [40]:
#interpolation method 3: Global Polynomial Interpolation (GPI)
arcpy.ga.GlobalPolynomialInterpolation(
    in_features = "NDAWN_stations",
    z_field = "Avg_Temp",
    out_raster = r"C:\Users\schil726\Downloads\GIS5571_Lab3\data\GPI"
)

<Result ''>

In [42]:
#perform interpolation methods for min temp

#interpolation method 1: inverse distance weighting (IDW)
arcpy.ddd.Idw(
    in_point_features = "NDAWN_stations",
    z_field = "MIN",
    out_raster = r"C:\Users\schil726\Downloads\GIS5571_Lab3\data\IDW_min"
)

#interpolation method 2: kriging
#set variables 
lagSize = 2000
majorRange = 2.6
partialSill = 542
nugget = 0
k_model = kModelOrdinary = KrigingModelOrdinary("SPHERICAL", lagSize,
                                majorRange, partialSill, nugget)

kriging_min = arcpy.sa.Kriging(
    in_point_features = "NDAWN_stations",
    z_field = "MIN",
    kriging_model = k_model,
)

#interpolation method 3: Global Polynomial Interpolation (GPI)
arcpy.ga.GlobalPolynomialInterpolation(
    in_features = "NDAWN_stations",
    z_field = "MIN",
    out_raster = r"C:\Users\schil726\Downloads\GIS5571_Lab3\data\GPI_min"
)

<Result ''>

In [43]:
#perform interpolation methods for max temp

#interpolation method 1: inverse distance weighting (IDW)
arcpy.ddd.Idw(
    in_point_features = "NDAWN_stations",
    z_field = "MAX",
    out_raster = r"C:\Users\schil726\Downloads\GIS5571_Lab3\data\IDW_max"
)

#interpolation method 2: kriging
#set variables 
lagSize = 2000
majorRange = 2.6
partialSill = 542
nugget = 0
k_model = kModelOrdinary = KrigingModelOrdinary("SPHERICAL", lagSize,
                                majorRange, partialSill, nugget)

kriging_max = arcpy.sa.Kriging(
    in_point_features = "NDAWN_stations",
    z_field = "MAX",
    kriging_model = k_model,
)

#interpolation method 3: Global Polynomial Interpolation (GPI)
arcpy.ga.GlobalPolynomialInterpolation(
    in_features = "NDAWN_stations",
    z_field = "MAX",
    out_raster = r"C:\Users\schil726\Downloads\GIS5571_Lab3\data\GPI_max"
)

<Result ''>